## Download Package

langchain, vector databasem enbedding model, pdfReader etc...

In [1]:
!pip install langchain langchain_openai chromadb transformers sentence-transformers pypdf langchain-community

  Using cached pypdf-5.4.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached openai-1.76.0-py3-none-any.whl.metadata (25 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 k

## PDF Load and page segmentation with langchain

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

## Load PDF page

In [3]:
loader = PyPDFLoader("제1주~제7주.pdf")
pages = loader.load_and_split()
print(len(pages))

337


첫번째 문서를 출력해봅시다. 랭체인으로 로드한 데이터의 형식은 다음을 따릅니다.  

`Document(metadata={'source': 파일명, 'page': 페이지 번호}, page_content='내용')`

## Chunking : document segmentation

In [4]:
# chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)
splited_docs = text_splitter.split_documents(pages)

print(len(splited_docs))

337


In [5]:
# first document
print('first document : ', splited_docs[0])

first document :  page_content='재미있고
유익한
부동산
상식
1주차 1차시
부동산의 의의' metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2024-09-01T13:31:19+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'User', 'moddate': '2024-09-01T13:31:19+09:00', 'source': '제1주~제7주.pdf', 'total_pages': 337, 'page': 0, 'page_label': '1'}


In [6]:
print('length of first document : ', len(splited_docs[0].page_content))

length of first document :  31


69개의 문서를 임베딩해야 합니다. 한글이 가능한 공개되어져 있는 오픈소스 임베딩을 사용.  

임베딩 모델 위치: https://huggingface.co/BAAI/bge-m3

In [7]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

/var/folders/rk/ggwwr7w16f35brvnmj0n9h9h0000gn/T/ipykernel_34157/2297800112.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/opt/anaconda3/envs/llm_study_python3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Chroma vector DB

In [8]:
chroma_db = Chroma.from_documents(splited_docs, HuggingFaceEmbeddings())

# similarity search
similar_docs = chroma_db.similarity_search("사용자의 질문")

/var/folders/rk/ggwwr7w16f35brvnmj0n9h9h0000gn/T/ipykernel_34157/1379591393.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  chroma_db = Chroma.from_documents(splited_docs, HuggingFaceEmbeddings())


In [ ]:
## Chroma 기반 pdf(docs 벡터화)
db = Chroma.from_documents(splited_docs, model_huggingface)

print('문서의 수:', db._collection.count())

In [ ]:
# 질의하기
question = "~~는 ~~ 인가요?"
docs = db.similarity_search(question, k=4)

In [ ]:
print('검색된 문서의 수 : ', len(docs))


In [ ]:
for doc in docs:
    print(doc)
    print('--' * 100)

## Save Vector DB in local file

In [ ]:
db_toFiles = Chroma.from_documents(splited_docs, model_huggingface, persist_directory = './realEstate.db')
print('문서의 수:', db_toFiles._collection.count())

In [ ]:
# 파일 확인
$ls -al

저장했던 DB file을 읽어와서 동일한 실습 가능

In [ ]:
db_fromfile = Chroma(persist_directory = './realEstate.db',embedding_function=model_huggingface)
print('문서의 수:', db_fromfile._collection.count())

## 질의하기
question = '부동산의 구성요소는?'
docs = db_fromfile.similarity_search(question, k=4)

print('검색된 문서의 수:', len(docs))

In [ ]:
question = '부동산의 구성요소는?'

# TOP 3개만 추출
docs3 = db_fromfile.similarity_search_with_relevance_scores(question, k=3)

In [ ]:
for doc in docs3:
  print(doc)
  print('--' * 100)

In [ ]:
print(f"가장 유사한 문서 : {docs3[0][0].page_content}")
print('*'*20)
print(f"가장 유사한 문서의 문서의 유사도 : {docs3[0][1]}")

# GPT-4 호출하기

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import os

In [ ]:
os.environ['OPENAI_API_KEY'] =  "여러분들의 Key 값"

In [ ]:
# GPT-4에 전달할 프롬프트
template = """당신은 재미있고 유익한 부동산 상식 강의자료를 설명해주는 챗봇 '삼성맨'입니다.
김태완 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 이모지를 사용하며 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

# 프롬프트 템플릿은 {question}에는 사용자의 질문이 들어간다. {context}에는 사용자의 질문에 따른 검색 결과가 들어간다.
prompt = PromptTemplate.from_template(template)

In [ ]:
# GPT-4 선언. 단, API Key 값이 셋팅되어져 있어야 실행 가능합니다.
llm = ChatOpenAI(model_name="gpt-4o")

In [ ]:
result = llm.invoke('안녕 반가워 GPT-4야 나는 김태완이라고해')
print(result.content)

## 벡터 데이터베이스와 LLM의 연결

`RetrievalQA.from_chain_type()`

- llm: 앞서 선언한 llm 객체를 넣어주세요.
- retriever: 앞서 선언한 retriever 객체를 넣어주세요.
- chain_type_kwargs={"prompt": ?}: 앞서 선언한 프롬프트 템플릿을 넣어주세요.

In [ ]:
# 벡터 데이터베이스를 LLM과 연결하기 위해서 retriever 객체로 선언
retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # LLM 연결
    retriever=retriever, # 리트리버(벡터 데이터베이스) 연결
    chain_type_kwargs={"prompt": prompt}, # 프롬프트 템플릿 연결
    return_source_documents=True # 실제 검색된 문서도 확인하려면 True
    )

In [ ]:
input_text = "부동산에서 토지란?"
chatbot_response = qa_chain.invoke(input_text)

In [ ]:
chatbot_response

In [ ]:
print('내가 넣었던 질문:', chatbot_response['query'])

In [ ]:
print('검색 결과를 바탕으로 GPT-4가 작성한 답변:', chatbot_response['result'])

In [ ]:
print('챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:')

for doc in chatbot_response['source_documents']:
  print(doc)
  print('--' * 100)

In [ ]:
input_text = "25년 2월의 서울 아파트 집값은?"
chatbot_response = qa_chain.invoke(input_text)

In [ ]:
chatbot_response['result']